In [ ]:
import matplotlib.pyplot as plt
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import math
%config InlineBackend.figure_format = 'retina'

In [ ]:
NSAMPLE = 1000
x_data = np.random.uniform(-10.5, 10.5, size=(1, NSAMPLE)).T
r_data = np.random.normal(size=(NSAMPLE, 1))
y_data = np.sin(0.75 * x_data) * 7.0 + x_data * 0.5 + r_data

In [ ]:
plt.figure(figsize=(8, 8))
plt.plot(x_data,y_data,'ro',alpha=0.3)
plt.show()

In [ ]:
# 输入
x = tf.placeholder(dtype=tf.float32, shape=[None,1])
# 需要逼近的目标值
y = tf.placeholder(dtype=tf.float32, shape=[None,1])

# 定义一个 20 个节点的隐藏层
hidden = tf.layers.dense(x, units=20, activation=tf.nn.tanh)
# 输出的预测值
y_out = tf.layers.dense(hidden, units=1, activation=None)

# 使用平方差的和作为损失值
loss = tf.nn.l2_loss(y_out - y)

# 定义优化器，目标是使损失值最小化
train_step = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)


# 初始化网络
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 训练 1000 轮
NEPOCH = 1000
for i in range(NEPOCH):
    sess.run(train_step, feed_dict={x: x_data, y: y_data})

In [ ]:
x_test = np.arange(-10.5, 10.5, 0.1)
x_test = np.reshape(x_test, newshape=(-1, 1))
y_test = sess.run(y_out, feed_dict={ x: x_test })

plt.figure(figsize=(8, 8))
plt.plot(x_data, y_data, 'b*', alpha=0.2)
plt.plot(x_test, y_test, 'r*')

plt.show()

In [ ]:
x_data, y_data = y_data, x_data
plt.figure(figsize=(8, 8))
plt.plot(x_data, y_data, 'b*', alpha=0.3)
plt.show()

In [ ]:
sess.run(tf.global_variables_initializer())

NEPOCH = 1000
for i in range(NEPOCH):
    sess.run(train_step, feed_dict={x: x_data, y: y_data})
    
x_test = np.arange(-10.5, 10.5, 0.1)
x_test = np.reshape(x_test, newshape=(-1, 1))
y_test = sess.run(y_out, feed_dict={ x: x_test })

plt.figure(figsize=(8, 8))
plt.plot(x_test, y_test, 'r*', x_data, y_data, 'b*', alpha=0.3)
plt.show()

In [ ]:
from scipy.stats import norm
sigma_values = [0.5, 1.0, 1.5]
linestyles = ['-', '--', ':']
mu_values = [1, -1, 0.3]
x = np.linspace(-10, 10, 1000)

plt.figure(figsize=(8, 8))

for mu, sigma, ls in zip(mu_values, sigma_values, linestyles):
    dist = norm(mu, sigma)

    plt.plot(x, dist.pdf(x), ls=ls,
             label='mu=%i, sigma=%.1f' % (mu, sigma))

plt.xlim(-5, 5)
plt.ylim(0, 0.85)

plt.xlabel('x')
plt.ylabel('p(x|mu,sigma)')
plt.title('Gaussian Distribution')

plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))

composite = None
for mu, sigma, ls in zip(mu_values, sigma_values, linestyles):
    dist = norm(mu, sigma).pdf(x)
    plt.plot(x, dist, ls=ls,
             label='mu=%i, sigma=%.1f' % (mu, sigma))
    if composite is not None:
        composite += dist
    else:
        composite = dist

plt.plot(x, composite, label='composite(unweighted)')
plt.xlim(-5, 5)
plt.ylim(0, 1.85)

plt.xlabel('x')
plt.ylabel('p(x|mu,sigma)')
plt.title('Gaussian Distribution')

plt.legend()
plt.show()

In [ ]:
tf.reset_default_graph()
x = tf.placeholder(shape=(None, 1), dtype=tf.float32, name='x')
y = tf.placeholder(shape=(None, 1), dtype=tf.float32, name='y')

mixture_size = 12
hidden1 = tf.layers.dense(x, units=64, activation=tf.nn.relu, name="hidden")
hidden2 = tf.layers.dense(hidden1, units=128, activation=tf.nn.relu, name="hidden1")
out = tf.layers.dense(hidden2, units=mixture_size*3, activation=None, name="out")
p, mu_out, sigma = tf.split(out,3,1)

# normalize to 0~1
p_out = tf.nn.softmax(p, name="prob_dist")
sigma_out = tf.exp(sigma, name='sigma')
factor = 1 / math.sqrt(2 * math.pi)

# Gaussian formulas
epsilon = 1e-5
tmp = - tf.square((y - mu_out)) / (2 * tf.square(tf.maximum(sigma_out, epsilon)))
y_normal = factor * tf.exp(tmp) / tf.maximum(sigma_out, epsilon)

In [ ]:
print(sigma_out)

In [ ]:
loss = tf.reduce_sum(tf.multiply(y_normal, p_out), axis=1, keep_dims=True)
loss = -tf.log(tf.maximum(loss, epsilon))
loss = tf.reduce_mean(loss)
train_step = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

NEPOCH = 2500
loss_vals = np.zeros(NEPOCH)
for i in range(NEPOCH):
    _, loss_val = sess.run([train_step, loss], feed_dict={x : x_data, y: y_data})
    loss_vals[i] = loss_val
    if i % 500 == 0:
        print('{}/{} loss: {}'.format(i, NEPOCH, loss_val))

plt.figure(figsize=(8, 8))
plt.plot(loss_vals)
plt.show()

In [ ]:
x_test = np.arange(-15, 15, 0.1)
x_test = np.reshape(x_test, newshape=(-1,1))
p_val, mu_val, sigma_val = sess.run([p_out, mu_out, sigma_out], feed_dict={x:x_test})

In [ ]:
M = 10
x_pred = np.repeat(x_test, M)

NTEST = len(x_test)
y_pred = np.random.rand(NTEST, M)
rn = np.random.randn(NTEST, M)

def get_pi_idx(x, pdf):
    N = pdf.size
    acc = 0
    for i in range(0, N):
        acc += pdf[i]
        if acc >= x:
            return i

for j in range(M):
    for i in range(NTEST):
        idx = get_pi_idx(y_pred[i, j], p_val[i])
        mu  = mu_val[i, idx]
        std = sigma_val[i, idx]
        y_pred[i, j] = mu + rn[i, j]*std

In [ ]:
plt.figure(figsize=(8, 8))
plt.plot(x_data, y_data, 'b*', x_test, y_pred, 'ro', alpha=0.3)
plt.show()